## **1. 라이브러리 설치 및 데이터 다운로드**

- 필요한 라이브러리를 설치하고, kaggle에서 데이터를 다운로드하여 준비한다.

In [ ]:
# 기본 라이브러리
import os
import zipfile
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import cv2
import yaml
import shutil
from tqdm import tqdm
from glob import glob
import random
import numpy as np
import torch

# YOLO 모델 및 시각화
from ultralytics import YOLO
from IPython.display import Image, display

# 한글 폰트 설정
import koreanize_matplotlib

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
!pip install koreanize-matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 92.4 MB/s eta 0:00:00


In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 97.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 109.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [ ]:
# kaggle.json 업로드
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"kimhaneulsky","key":"fd6a63fd32f0e60daee70e35a4e8b2ea"}'}

In [ ]:
# Kaggle API Key 설정
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!pip install -q kaggle

In [ ]:
# 데이터 다운로드 및 압축 해제
!kaggle competitions download -c ai02-level1-project
!unzip -q ai02-level1-project.zip -d ./ai02_data

100% 3.91G/3.91G [00:12<00:00, 292MB/s]
100% 3.91G/3.91G [00:12<00:00, 349MB/s]


## **2. 시드 고정**
- 재현 가능한 결과를 위해 random, numpy, torch의 시드를 고정한다.
- 이를 통해 매 실행마다 동일한 데이터 셔플링, weight 초기화 등을 보장받을 수 있다.

In [ ]:
random.seed(71)
np.random.seed(71)
torch.manual_seed(71)
torch.cuda.manual_seed(71)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

## **3. 데이터 경로 설정**

- 데이터 디렉토리 구조를 기반으로, 학습 이미지, 테스트 이미지, 어노테이션 파일의 경로를 지정한다.



In [ ]:
# 데이터 경로
data_path = '/content/ai02_data'

# 훈련, 검증, 테스트 데이터 경로 설정
train_dir = os.path.join(data_path, 'train_images')
test_dir = os.path.join(data_path, 'test_images')
annotation_dir = os.path.join(data_path, 'train_annotations')

## **4. 어노테이션 txt 파일 변환**
- COCO 형식의 json 어노테이션 파일을 YOLO 학습에 필요한 .txt 파일 포맷으로 변환한다.

| 항목     | COCO JSON 형식            | YOLO `.txt` 형식                         |
| ------ | ----------------------- | -------------------------------------- |
| 저장 방식  | 하나의 `.json`에 여러 이미지 포함  | 이미지별 `.txt` 파일 개별 생성                   |
| 바운딩 박스 | `[x, y, width, height]` | `class x_center y_center width height` |
| 좌표 단위  | 절대 픽셀 기준                | 이미지 크기로 정규화된 상대 좌표 (0 \~ 1)            |

  ① train_annotations/ 폴더 내 모든 .json 파일을 순회한다.

  ② 각 이미지의 width, height를 기준으로 bounding box 좌표를 YOLO 형식으로 정규화한다.

  ③ 클래스 이름은 정렬 후 index 부여 방식으로 class_id로 변환된다.

  ④ 변환된 결과는 이미지 파일명과 동일한 .txt 파일로 저장되며, 저장 위치는 `/content/ai02_data/labels/train/`이다.

In [ ]:
annotation_dir = '/content/ai02_data/train_annotations'
save_dir = '/content/ai02_data/labels/train'
os.makedirs(save_dir, exist_ok=True)

all_category_names = set()
for root,_, files in os.walk(annotation_dir):
    for file in files:
        if file.endswith('.json'):
            with open(os.path.join(root, file), 'r', encoding='utf-8') as f:
                data = json.load(f)
                for cat in data['categories']:
                    all_category_names.add(cat['name'])

category_name_to_id = {name: idx for idx, name in enumerate(sorted(all_category_names))}

for root,_, files in os.walk(annotation_dir):
    for file in files:
        if file.endswith('.json'):
            json_path = os.path.join(root, file)
            with open(json_path, 'r', encoding='utf-8') as f:
                data = json.load(f)

            images = {img['id']: img for img in data['images']}
            categories = {cat['id']: category_name_to_id[cat['name']] for cat in data['categories']}

            for ann in tqdm(data['annotations'], desc=f'Processing {file}'):
                img_info = images[ann['image_id']]
                img_name = img_info['file_name']
                w, h = img_info['width'], img_info['height']

                bbox = ann['bbox']
                x_center = (bbox[0] + bbox[2] / 2) / w
                y_center = (bbox[1] + bbox[3] / 2) / h
                bw = bbox[2] / w
                bh = bbox[3] / h

                class_id = categories[ann['category_id']]
                txt_filename = os.path.splitext(img_name)[0] + '.txt'
                txt_path = os.path.join(save_dir, txt_filename)

                with open(txt_path, 'a') as f:
                    f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {bw:.6f} {bh:.6f}\n")

Processing K-003483-020238-027733-031885_0_2_0_2_75_000_200.json: 100%|██████████| 1/1 [00:00<00:00, 2761.23it/s]
Processing K-003483-020238-027733-031885_0_2_0_2_90_000_200.json: 100%|██████████| 1/1 [00:00<00:00, 3223.91it/s]
Processing K-003483-020238-027733-031885_0_2_0_2_70_000_200.json: 100%|██████████| 1/1 [00:00<00:00, 2520.62it/s]
Processing K-003483-020238-027733-031885_0_2_0_2_75_000_200.json: 100%|██████████| 1/1 [00:00<00:00, 2882.68it/s]
Processing K-003483-020238-027733-031885_0_2_0_2_90_000_200.json: 100%|██████████| 1/1 [00:00<00:00, 2757.60it/s]
Processing K-003483-020238-027733-031885_0_2_0_2_70_000_200.json: 100%|██████████| 1/1 [00:00<00:00, 2621.44it/s]
Processing K-003483-020238-027733-031885_0_2_0_2_75_000_200.json: 100%|██████████| 1/1 [00:00<00:00, 2820.65it/s]
Processing K-003483-020238-027733-031885_0_2_0_2_70_000_200.json: 100%|██████████| 1/1 [00:00<00:00, 2833.99it/s]
Processing K-003483-020238-027733-031885_0_2_0_2_75_000_200.json: 100%|██████████| 1/1 [

In [ ]:
!ls /content/ai02_data/labels/train | head

K-001900-010224-016551-031705_0_2_0_2_70_000_200.txt
K-001900-010224-016551-031705_0_2_0_2_75_000_200.txt
K-001900-010224-016551-031705_0_2_0_2_90_000_200.txt
K-001900-010224-016551-033009_0_2_0_2_70_000_200.txt
K-001900-010224-016551-033009_0_2_0_2_75_000_200.txt
K-001900-010224-016551-033009_0_2_0_2_90_000_200.txt
K-001900-016548-018110-021026_0_2_0_2_70_000_200.txt
K-001900-016548-018110-021026_0_2_0_2_75_000_200.txt
K-001900-016548-018110-021026_0_2_0_2_90_000_200.txt
K-001900-016548-018110-027926_0_2_0_2_70_000_200.txt


In [ ]:
with open('/content/ai02_data/labels/train/K-001900-010224-016551-031705_0_2_0_2_70_000_200.txt') as f:
    print(f.read())

4 0.262807 0.262500 0.251025 0.332813
9 0.725922 0.278125 0.292008 0.351562
26 0.768443 0.732812 0.215164 0.123438



## **5. train/val 분리**

- 모델 학습의 성능을 정확히 평가하기 위해, 학습 이미지 및 라벨을 Train/Validation으로 8:2 비율로 분리한다.

In [ ]:
# 원본 이미지/라벨 경로
src_img_dir = os.path.join(data_path, 'train_images')
src_label_dir = os.path.join(data_path, 'labels/train')

# train/val 분리 경로
train_img_dir = os.path.join(data_path, 'images/train')
val_img_dir = os.path.join(data_path, 'images/val')
train_label_dir = os.path.join(data_path, 'labels/train_split')
val_label_dir = os.path.join(data_path, 'labels/val')

# 디렉토리 생성
for d in [train_img_dir, val_img_dir, train_label_dir, val_label_dir]:
    os.makedirs(d, exist_ok=True)

# 이미지 리스트 섞고 분할
image_paths = sorted(glob.glob(os.path.join(src_img_dir, '*.png')))
random.shuffle(image_paths)
val_count = int(len(image_paths) * 0.2)
val_imgs = image_paths[:val_count]
train_imgs = image_paths[val_count:]

# 복사 함수 정의
def copy_data(img_list, dst_img_dir, dst_label_dir):
    for img_path in img_list:
        file_name = os.path.basename(img_path)
        label_name = file_name.replace('.png', '.txt')
        label_path = os.path.join(src_label_dir, label_name)

        shutil.copy(img_path, os.path.join(dst_img_dir, file_name))
        if os.path.exists(label_path):
            shutil.copy(label_path, os.path.join(dst_label_dir, label_name))

# 복사 실행
copy_data(train_imgs, train_img_dir, train_label_dir)
copy_data(val_imgs, val_img_dir, val_label_dir)

print(f"✅ Train 이미지 수: {len(train_imgs)}")
print(f"✅ Val 이미지 수: {len(val_imgs)}")

✅ Train 이미지 수: 1192
✅ Val 이미지 수: 297


## **6. yaml 파일 생성**
- YOLOv8 모델 학습을 위해 train/val 이미지 경로와 클래스 이름 정보를 포함한 data.yaml 파일을 생성한다.

In [ ]:
yaml_path = '/content/ai02_data/data.yaml'

# 모든 카테고리 이름 수집용 딕셔너리
category_id_to_name = {}

for root, _, files in os.walk(annotation_dir):
    for file in files:
        if file.endswith('.json'):
            json_path = os.path.join(root, file)
            with open(json_path, 'r', encoding='utf-8') as f:
                data = json.load(f)

            for cat in data.get('categories', []):
                category_id_to_name[cat['id']] = cat['name']

# category_id를 정렬된 순서대로 클래스 인덱스 매핑
sorted_ids = sorted(category_id_to_name.keys())
class_names = [category_id_to_name[cat_id] for cat_id in sorted_ids]

yaml_dict = {
    'train': '/content/ai02_data/images/train',
    'val': '/content/ai02_data/images/val',
    'nc': len(class_names),
    'names': class_names
}

# YAML 저장
with open(yaml_path, 'w', encoding='utf-8') as f:
    yaml.dump(yaml_dict, f, allow_unicode=True)

print(f"✅ data.yaml 저장 완료!\n클래스 수: {len(class_names)}\n클래스 목록: {class_names}")

✅ data.yaml 저장 완료!
클래스 수: 73
클래스 목록: ['보령부스파정 5mg', '뮤테란캡슐 100mg', '일양하이트린정 2mg', '기넥신에프정(은행엽엑스)(수출용)', '무코스타정(레바미피드)(비매품)', '알드린정', '뉴로메드정(옥시라세탐)', '타이레놀정500mg', '에어탈정(아세클로페낙)', '삼남건조수산화알루미늄겔정', '타이레놀이알서방정(아세트아미노펜)(수출용)', '삐콤씨에프정 618.6mg/병', '조인스정 200mg', '쎄로켈정 100mg', '리렉스펜정 300mg/PTP', '아빌리파이정 10mg', '자이프렉사정 2.5mg', '다보타민큐정 10mg/병', '써스펜8시간이알서방정 650mg', '에빅사정(메만틴염산염)(비매품)', '리피토정 20mg', '크레스토정 20mg', '가바토파정 100mg', '동아가바펜틴정 800mg', '오마코연질캡슐(오메가-3-산에틸에스테르90)', '란스톤엘에프디티정 30mg', '리리카캡슐 150mg', '종근당글리아티린연질캡슐(콜린알포세레이트)\xa0', '콜리네이트연질캡슐 400mg', '트루비타정 60mg/병', '스토가정 10mg', '노바스크정 5mg', '마도파정', '플라빅스정 75mg', '엑스포지정 5/160mg', '펠루비정(펠루비프로펜)', '아토르바정 10mg', '라비에트정 20mg', '리피로우정 20mg', '자누비아정 50mg', '맥시부펜이알정 300mg', '메가파워정 90mg/병', '쿠에타핀정 25mg', '비타비백정 100mg/병', '놀텍정 10mg', '자누메트정 50/850mg', '큐시드정 31.5mg/PTP', '아모잘탄정 5/100mg', '세비카정 10/40mg', '트윈스타정 40/5mg', '카나브정 60mg', '울트라셋이알서방정', '졸로푸트정 100mg', '트라젠타정(리나글립틴)', '비모보정 500/20mg', '레일라정', '리바로정 4mg', '렉사프로정 15mg', '트라젠타듀오정 2.5/850mg', '낙소졸정 5

## **7. 모델 훈련 및 검증 평가**


### (1) YOLOv8n 모델 학습

- 초기 실험 단계에서는 최신 YOLO 모델 중에서 가볍고 학습 속도가 빠른 YOLOv8n (Nano) 구조를 선택하였다.

- YOLOv8은 Ultralytics에서 제공하는 객체 탐지 모델 중 최적화와 성능이 균형을 이루는 최신 버전이다.

- 본 실험에 사용된 주요 학습 설정은 다음과 같다.

| 항목     | 값              |
| ------ | -------------- |
| 모델 구조  | YOLOv8n (Nano) |
| 입력 사이즈 | 640 x 640      |
| 에폭 수   | 20             |
| 옵티마이저  | Adam           |
| 배치 크기  | 16             |
| 평가 지표  | mAP\@0.5       |

In [ ]:
model = YOLO("yolov8n.pt")

model.train(
    data='/content/ai02_data/data.yaml',
    epochs=20,
    imgsz=640,
    batch=16,
    device=0,
    workers=4,                 # 데이터 로딩 멀티스레딩
    patience=5,                # early stopping: 개선 없으면 조기 종료
    optimizer='Adam',          # 옵티마이저
    seed=71,                   # 시드 고정으로 재현성 확보
    save=True,                 # 모델 체크포인트 저장 (기본 True)
    save_period=5,             # 매 5 에폭마다 저장
    project="pill_yolo_exp",   # 프로젝트 폴더명
    name="yolov8n_base",       # 실험 이름
    verbose=True               # 학습 로그 출력
)

Ultralytics 8.3.146 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/ai02_data/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8n_base2, nbs=64, nms=False, opset=None, optimize=False, optimizer=Adam, overlap_mask=True, patience=5, perspective=0.0, plots=True, pose=12.0, pretra

train: Scanning /content/ai02_data/labels/train... 1192 images, 0 backgrounds, 2 corrupt: 100%|██████████| 1192/1192 [00:02<00:00, 545.55it/s]

train: /content/ai02_data/images/train/K-003351-016262-018357_0_2_0_2_75_000_200.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     6.8878]
train: /content/ai02_data/images/train/K-003544-004543-012247-016551_0_2_0_2_70_000_200.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     7.0293]
train: New cache created: /content/ai02_data/labels/train.cache


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2653.4±1616.8 MB/s, size: 1715.1 KB)


val: Scanning /content/ai02_data/labels/val... 297 images, 0 backgrounds, 0 corrupt: 100%|██████████| 297/297 [00:00<00:00, 479.42it/s]

...

                   all        297        910      0.712      0.979       0.85      0.834



20 epochs completed in 0.083 hours.
Optimizer stripped from pill_yolo_exp/yolov8n_base2/weights/last.pt, 6.4MB
Optimizer stripped from pill_yolo_exp/yolov8n_base2/weights/best.pt, 6.4MB

Validating pill_yolo_exp/yolov8n_base2/weights/best.pt...
Ultralytics 8.3.146 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
Model summary (fused): 72 layers, 3,091,487 parameters, 0 gradients, 8.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:02<00:00,  4.19it/s]


                   all        297        910      0.712      0.979      0.849      0.833
            보령부스파정 5mg         33         33      0.771          1      0.938      0.938
           뮤테란캡슐 100mg          2          2      0.309          1      0.448      0.428
           일양하이트린정 2mg          7          7      0.774          1      0.774      0.774
    기넥신에프정(은행엽엑스)(수출용)         95         95      0.821          1      0.888      0.877
     무코스타정(레바미피드)(비매품)          4          4      0.461          1       0.67      0.648
                  알드린정         10         10      0.599          1      0.844      0.844
          뉴로메드정(옥시라세탐)          7          7      0.931          1      0.995      0.954
            타이레놀정500mg          2          2      0.665      0.996      0.828      0.762
          에어탈정(아세클로페낙)          9          9      0.512          1      0.617      0.565
         삼남건조수산화알루미늄겔정         31         31      0.824          1      0.886      0.882
타이레놀이알서방정(아세트아미노펜)(수출

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fdac03ebd10>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027

(2) YOLOv8n 검증평가

- 학습이 완료된 YOLOv8n 모델을 검증 데이터셋(Validation Set)에 적용하여 성능을 평가하였다.
- Precision과 Recall이 균형 있게 높고, mAP@0.5 및 mAP@0.5:0.95 점수 모두 우수한 결과를 보였다.

| 항목            | 값     |
| ------------- | ----- |
| Precision     | 0.712 |
| Recall        | 0.979 |
| mAP\@0.5      | 0.849 |
| mAP\@0.5:0.95 | 0.834 |
| 이미지 수         | 297장  |
| 객체 수          | 910개  |

In [ ]:
metrics = model.val(data='/content/ai02_data/data.yaml')
print(metrics.box.map)  # mAP@0.5

Ultralytics 8.3.146 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 4203.3±904.0 MB/s, size: 1693.5 KB)


val: Scanning /content/ai02_data/labels/val.cache... 297 images, 0 backgrounds, 0 corrupt: 100%|██████████| 297/297 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:03<00:00,  4.82it/s]


                   all        297        910      0.712      0.979      0.849      0.834
            보령부스파정 5mg         33         33      0.771          1      0.938      0.938
           뮤테란캡슐 100mg          2          2      0.309          1      0.448      0.428
           일양하이트린정 2mg          7          7      0.773          1      0.774      0.774
    기넥신에프정(은행엽엑스)(수출용)         95         95      0.821          1       0.89      0.883
     무코스타정(레바미피드)(비매품)          4          4      0.461          1       0.67      0.648
                  알드린정         10         10      0.599          1      0.844      0.844
          뉴로메드정(옥시라세탐)          7          7       0.93          1      0.995      0.954
            타이레놀정500mg          2          2      0.665      0.994      0.828      0.762
          에어탈정(아세클로페낙)          9          9      0.512          1      0.619      0.567
         삼남건조수산화알루미늄겔정         31         31      0.824          1      0.886      0.882
타이레놀이알서방정(아세트아미노펜)(수출

## **8. 모델 평가**
- 학습한 YOLOv8n 모델의 best weight를 불러와 테스트 이미지에 대한 객체 탐지를 수행한다.
- 예측 결과는 .txt 형식의 라벨 파일과 함께 시각화된 이미지로 저장된다.

In [ ]:
model = YOLO("/content/pill_yolo_exp/yolov8n_base2/weights/best.pt")

results = model.predict(
    source=test_dir,
    imgsz=640,
    conf=0.25,       # confidence threshold
    save=True,       # 이미지에 결과 시각화 저장
    save_txt=True,   # YOLO 포맷으로 예측 결과 저장
    project='pill_yolo_exp',
    name='test_results',
    exist_ok=True    # 이미 폴더 있어도 덮어쓰기 허용
)


image 1/843 /content/ai02_data/test_images/1.png: 640x512 1 뉴로메드정(옥시라세탐), 1 삼남건조수산화알루미늄겔정, 1 리리카캡슐 150mg, 1 세비카정 10/40mg, 70.4ms
image 2/843 /content/ai02_data/test_images/10.png: 640x512 1 보령부스파정 5mg, 1 타이레놀이알서방정(아세트아미노펜)(수출용), 1 조인스정 200mg, 1 리리카캡슐 150mg, 10.4ms
image 3/843 /content/ai02_data/test_images/100.png: 640x512 1 보령부스파정 5mg, 1 삐콤씨에프정 618.6mg/병, 1 리리카캡슐 150mg, 1 엑스포지정 5/160mg, 9.6ms
image 4/843 /content/ai02_data/test_images/1003.png: 640x512 1 기넥신에프정(은행엽엑스)(수출용), 1 리피토정 20mg, 1 비모보정 500/20mg, 1 로수바미브정 10/20mg, 8.4ms
image 5/843 /content/ai02_data/test_images/1004.png: 640x512 1 기넥신에프정(은행엽엑스)(수출용), 1 리피토정 20mg, 1 비모보정 500/20mg, 1 로수바미브정 10/20mg, 9.0ms
image 6/843 /content/ai02_data/test_images/1005.png: 640x512 1 기넥신에프정(은행엽엑스)(수출용), 1 리피토정 20mg, 1 비모보정 500/20mg, 1 로수바미브정 10/20mg, 9.8ms
image 7/843 /content/ai02_data/test_images/1006.png: 640x512 1 기넥신에프정(은행엽엑스)(수출용), 1 라비에트정 20mg, 1 울트라셋이알서방정, 1 케이캡정 50mg, 8.6ms
image 8/843 /content/ai02_data/test_images/1007.png: 640x512 1